In [1]:
# Importa as dependências
import numpy as np
import cv2

In [2]:
# Transforma uma imagem do domínio do espaço para o da frequência 
def space_to_freq (image):

    # Converte a imagem para floats e aplica o DFT, obtendo os valores complexos
    dft = cv2.dft(np.float32(image), flags = cv2.DFT_COMPLEX_OUTPUT)

    # Realiza um 'shift' do canto superior esquerdo para o centro da matriz na frequência
    dft_shift = np.fft.fftshift(dft)

    # Extrai a magnitude e fase da matrix na frequência
    mag, phase = cv2.cartToPolar(dft_shift[:, :, 0], dft_shift[:, :, 1])

    # Transforma de magnitude para espectro
    spec = np.log(mag) / 20

    # Retorna os valores calculados
    return spec, mag, phase

In [3]:
# Transforma uma imagem do domínio da frequência para o do espaço
def freq_to_space(spec, mag, phase, img_min, img_max):

    # Traduz do espectro para magnitude
    mag = np.exp(spec * 20)

    # Converte a magnitude e fase para valores reais e complexos no plano cartesiano
    real, imag = cv2.polarToCart(mag, phase)

    # Combina os componentes carteziados em uma matriz de valores complexos
    back = cv2.merge([real, imag])

    # Realiza um 'shift' do centro para o canto superior esquerdo
    back_ishift = np.fft.ifftshift(back)

    # Realiza uma IDFT e salva os complexos resultantes
    img_back = cv2.idft(back_ishift)

    # Combina os componentes complexos de volta para a imagem
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Re-normaliza para o range uint8, igual ao original
    min, max = np.amin(img_back, (0, 1)), np.amax(img_back, (0, 1))
    image = cv2.normalize(img_back, None, alpha=img_min, 
                            beta=img_max, norm_type=cv2.NORM_MINMAX, 
                            dtype=cv2.CV_8U)

    # Retorna a imagem
    return image

In [4]:
# Função de callback do mouse
pt1_x, pt1_y, drawing, color, pointer_size, pointer_type = 0, 0, 0, 0, 1, 1

def line_drawing(event, x, y, flags, param):
    
    # Referenciar variáveis globais para execução
    global sidebyside, spectrum_image, pt1_x, pt1_y, drawing, color, pointer_size, pointer_type

    # Caso no modo Linha
    if pointer_type == 1:
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            pt1_x, pt1_y = x, y
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            cv2.line(spectrum_image, (pt1_x, pt1_y), (x, y), color=(color, color, color), thickness=pointer_size)
            pt1_x, pt1_y = x, y
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            cv2.line(spectrum_image, (pt1_x, pt1_y), (x, y), color=(color, color, color), thickness=pointer_size)

    # Caso no modo Círculo
    elif pointer_type == 2:
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            pt1_x, pt1_y = x, y
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            radius = int(np.sqrt((pt1_x - x)**2 + (pt1_y - y)**2))
            cv2.circle(spectrum_image, (pt1_x, pt1_y), radius, color=(color, color, color), thickness=-1)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            radius = int(np.sqrt((pt1_x - x)**2 + (pt1_y - y)**2))
            cv2.circle(spectrum_image, (pt1_x, pt1_y), radius, color=(color, color, color), thickness=-1)

    # Caso no modo Anticírculo
    elif pointer_type == 3:
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            pt1_x, pt1_y = x, y
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            radius = int(np.sqrt((pt1_x - x)**2 + (pt1_y - y)**2))
            mask = np.ones(spectrum_image.shape[:2], dtype=np.uint8) * 255
            cv2.circle(mask, (pt1_x, pt1_y), radius, color=(255, 255, 255), thickness=5)
            spectrum_image = cv2.bitwise_and(spectrum_image, spectrum_image, mask=mask)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            radius = int(np.sqrt((pt1_x - x)**2 + (pt1_y - y)**2))
            mask = np.zeros(spectrum_image.shape[:2], dtype=np.uint8)
            cv2.circle(mask, (pt1_x, pt1_y), radius, color=(1, 1, 1), thickness=-1)
            spectrum_image = cv2.bitwise_and(spectrum_image, spectrum_image, mask=mask)

In [5]:
# Realiza o calculo da taxa de compressão
def calc_compression_ratio(image, size):
    
    # Calcula a quantidade de pixes apagados
    zeros = size - np.count_nonzero(image)
    ratio = (zeros / size) * 100
    return ratio

In [6]:
# Call-back function para o menu de ferramentas
def menu_controller(event, x, y, flags, param):

    # Referenciar variáveis globais para execução
    global menu_image, color, pointer_size, pointer_type

    # Ao soltar, atualiza algum parâmetro da ferramenta
    if event == cv2.EVENT_LBUTTONUP:
        
        # Atualização da cor do pincel
        if y > 320:
            if x < 50: color = 0
            elif x > 450: color = 1
            else: color = (x + 50) / 500

        # Atualização do tamanho do pincel
        elif y > 250:
            if x < 55: pointer_size = 0.1
            elif x > 450: pointer_size = 1
            else: pointer_size = (x + 50) / 500

            # Ajuda para escala 0 -> 10
            pointer_size *= 10
            pointer_size = int(pointer_size)

        # Atualização do tipo do pincel
        else:
            if x > 333: 
                pointer_type = 3  # Anticírculo
            elif x > 166: 
                pointer_type = 2  # Círculo
            else: 
                pointer_type = 1  # Linha

In [7]:
# Lê a imagem em escala de cinza
image = cv2.imread('images/arara.jpg', 0)
menu_image = cv2.imread('./icons/design.png')

# Obtém os valores de min e max da imagem
img_min, img_max = np.amin(image, (0, 1)), np.amax(image, (0, 1))

# Transforma para o domínio da frequência
spectrum_image, mag, phase = space_to_freq(image)
saved = spectrum_image

# Cria a janela e seta o callback
cv2.namedWindow('Spectrum')
cv2.setMouseCallback('Spectrum', line_drawing)

# Cria a janela e seta o callback
cv2.namedWindow('Menu')
cv2.setMouseCallback('Menu', menu_controller)
cv2.imshow('Menu', menu_image)

while(1):

    # Realiza um update na imagem resultante
    freq_image = freq_to_space(spectrum_image, mag, phase, img_min, img_max)

    # Calcula a taxa de compressão da imagem
    new_ratio = calc_compression_ratio(spectrum_image, 
            np.array(np.shape(spectrum_image)).prod())

    # Mostra a taxa de compressão na imagem
    pointer_type_str = {1: 'pencil', 2: 'circle', 3: 'anticircle'}[pointer_type]
    cv2.putText(freq_image, f"Compression: {new_ratio:.2f}%",
                (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(freq_image, f"Tool: {pointer_type_str}",
                (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    
    # Concatena a imagem modificada com sua versão na frequência
    sidebyside = np.concatenate((spectrum_image, freq_image / 255), axis=1)       

    # Mostra a imagem montada
    cv2.imshow('Spectrum', sidebyside)

    # Para a execução caso Q seja pressionado
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Reseta as edições caso R seja pressionado
    elif cv2.waitKey(1) & 0xFF == ord('r'):
        spectrum_image, mag, phase = space_to_freq(image)

# Fecha as janelas
cv2.destroyAllWindows()